In [237]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/carvana-predict-car-prices/carvana.csv


In [238]:
data=pd.read_csv("/kaggle/input/carvana-predict-car-prices/carvana.csv")

In [239]:
data.head()

,Name,Year,Miles,Price
0,Chevrolet Trax,2018,41946,16990
1,GMC Terrain,2020,45328,23990
2,Jeep Wrangler,2012,81068,21590
3,Jeep Renegade,2019,35372,21590
4,BMW X,20173,68992,22990


In [240]:
data.columns.unique()

Index(['Name', 'Year', 'Miles', 'Price'], dtype='object')

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    22000 non-null  object
 1   Year    22000 non-null  int64 
 2   Miles   22000 non-null  int64 
 3   Price   22000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 687.6+ KB


In [242]:
data.isnull().sum()

Name     0
Year     0
Miles    0
Price    0
dtype: int64

In [243]:
data.describe()

,Year,Miles,Price
count,2.200000e+04,22000.000000,22000.000000
mean,1.946648e+05,54445.697318,20707.285136
std,1.630370e+06,25685.933926,6535.580304
min,2.009000e+03,53.000000,10990.000000
25%,2.014000e+03,35512.000000,16590.000000
50%,2.017000e+03,51890.000000,19590.000000
75%,2.020000e+03,73191.000000,22990.000000
max,2.022350e+07,120167.000000,102990.000000


In [244]:
col=['Name', 'Year']
for i in col:
    print(i,": ")
    print(data[i].unique())

Name : 
[' Chevrolet Trax' ' GMC Terrain' ' Jeep Wrangler' ' Jeep Renegade'
 ' BMW X' ' Buick Encore' ' Jeep Compass' ' Jeep Patriot'
 ' Chrysler Sebring' ' Subaru Legacy' ' Subaru XV Crosstrek'
 ' Mitsubishi Eclipse Cross' ' Toyota Corolla' ' Nissan Pathfinder'
 ' Lincoln MKC' ' Lexus ES' ' Dodge Grand Caravan Passenger'
 ' Ford Escape' ' Mitsubishi Mirage G' ' Toyota C-HR' ' Toyota Yaris'
 ' MAZDA CX-' ' Nissan Rogue Sport' ' Volkswagen Atlas' ' MAZDA MAZDA'
 ' Nissan Sentra' ' Honda Civic' ' Nissan Rogue' ' Toyota Corolla iM'
 ' Subaru Forester' ' Volkswagen Jetta' ' FIAT ' ' Honda Accord'
 ' Hyundai Sonata' ' GMC Acadia' ' Kia Optima' ' Hyundai Veloster'
 ' Hyundai Elantra' ' FIAT  Spider' ' Honda Odyssey' ' Volkswagen Tiguan'
 ' Acura RDX' ' Jeep Grand Cherokee' ' Chrysler Voyager' ' Hyundai Accent'
 ' Honda CR-V' ' INFINITI QX' ' Chevrolet Traverse'
 ' Mitsubishi Outlander Sport' ' Hyundai Kona' ' Nissan Frontier King Cab'
 ' Chevrolet Spark' ' Nissan Altima' ' Kia Soul' ' Ford F

In [245]:
data['Name'].unique().size

416

In [246]:
data['Year'].unique().size

211

**While looking at the year we can find many error. It combines with date and month in many areas. Let us resolve it.**

In [247]:
df=data.copy()

In [248]:
df.head()

,Name,Year,Miles,Price
0,Chevrolet Trax,2018,41946,16990
1,GMC Terrain,2020,45328,23990
2,Jeep Wrangler,2012,81068,21590
3,Jeep Renegade,2019,35372,21590
4,BMW X,20173,68992,22990


In [249]:
year_d=df['Year'].astype('str')

In [250]:
year_d

0         2018
1         2020
2         2012
3         2019
4        20173
         ...  
21995     2014
21996     2016
21997     2013
21998     2020
21999     2012
Name: Year, Length: 22000, dtype: object

In [251]:
year_d[0][0:4]

'2018'

In [252]:
year_d[4][0:4]

'2017'

In [253]:
year=[]
for y in year_d:
    year.append(y[0:4])

In [254]:
df.drop(['Year'],inplace=True,axis=1)

In [255]:
df.insert(loc=1, column='Year', value=year)

In [256]:
df.head()

,Name,Year,Miles,Price
0,Chevrolet Trax,2018,41946,16990
1,GMC Terrain,2020,45328,23990
2,Jeep Wrangler,2012,81068,21590
3,Jeep Renegade,2019,35372,21590
4,BMW X,2017,68992,22990


In [257]:
df['Year']=df['Year'].apply(pd.to_numeric)

In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    22000 non-null  object
 1   Year    22000 non-null  int64 
 2   Miles   22000 non-null  int64 
 3   Price   22000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 687.6+ KB


In [259]:
df['Year'].unique().size

15

**We removed those errors from year. Converting 211 to 15 unique years.**

In [260]:
from sklearn.preprocessing import LabelEncoder

In [261]:
le=LabelEncoder()

In [262]:
df['Name']=le.fit_transform(df['Name'])

In [263]:
df.head()

,Name,Year,Miles,Price
0,71,2018,41946,16990
1,127,2020,45328,23990
2,195,2012,81068,21590
3,193,2019,35372,21590
4,20,2017,68992,22990


In [264]:
df1=df.copy()

In [265]:
df1.head()

,Name,Year,Miles,Price
0,71,2018,41946,16990
1,127,2020,45328,23990
2,195,2012,81068,21590
3,193,2019,35372,21590
4,20,2017,68992,22990


In [266]:
x=df1.iloc[:,0:3]

In [267]:
y=df1.iloc[:,-1:]

In [268]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [269]:
from sklearn.metrics import r2_score

In [270]:
from sklearn.ensemble import RandomForestRegressor

In [271]:
regr = RandomForestRegressor(n_estimators=100,n_jobs=-1)

In [272]:
model=regr.fit(x_train,y_train.values.ravel())

In [273]:
y_pred=regr.predict(x_test)

In [274]:
r2_score(y_test,y_pred)

0.8557354050828971

**Through Random Forest Regressor, we get score up to 85%.**